In [1]:
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape

In [2]:
API_KEY = "ea9637fd9f0c41f3e2e932faa99dfcd76f8041aa"

In [3]:
def fetch_state_data():
    """Fetch state-level Census data, return as DataFrame."""
    state_url = (
        f"https://api.census.gov/data/2019/pep/population"
        f"?get=NAME,POP&for=state:*&key={API_KEY}"
    )
    r_states = requests.get(state_url)
    state_data = r_states.json()
    df = pd.DataFrame(state_data[1:], columns=state_data[0])
    df["POP"] = df["POP"].astype(int)
    return df

In [4]:
def fetch_county_data():
    """Fetch county-level Census data, return as DataFrame."""
    county_url = (
        f"https://api.census.gov/data/2019/pep/population"
        f"?get=NAME,POP&for=county:*&key={API_KEY}"
    )
    r_counties = requests.get(county_url)
    county_data = r_counties.json()
    df = pd.DataFrame(county_data[1:], columns=county_data[0])
    df["POP"] = df["POP"].astype(int)
    # Split the NAME column into state and county
    df[['countyName', 'stateName']] = df['NAME'].str.split(',', expand=True)
    # Combine state and county to form FIPS
    df["FIPS"] = df["state"].str.zfill(2) + df["county"].str.zfill(3)
    return df

In [5]:
def fetch_geojson(url: str):
    """Cache the geojson data from the provided URL."""
    return requests.get(url).json()

In [6]:
def build_states_gdf(state_df, state_abbrev_to_fips):
    """Build GeoDataFrame for US states."""
    state_pop_dict = state_df.set_index("state")["POP"].to_dict()
    state_name_dict = state_df.set_index("state")["NAME"].to_dict()
    url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"
    geo_data = fetch_geojson(url)
    rows = []
    for feat in geo_data["features"]:
        abbrev = feat["id"]
        geom = shape(feat["geometry"])
        fips = state_abbrev_to_fips.get(abbrev)
        if fips:
            pop_val = state_pop_dict.get(fips, "No data")
            name_val = state_name_dict.get(fips, "No data")
            rows.append({
                "geometry": geom,
                "STATE_FIPS": fips,
                "NAME": name_val,
                "POP": pop_val
            })
    return gpd.GeoDataFrame(rows, crs="EPSG:4326")

In [7]:
def build_counties_gdf(county_df):
    """Build GeoDataFrame for US counties and simplify geometries for performance."""
    county_pop_dict = county_df.set_index("FIPS")["POP"].to_dict()
    county_name_dict = county_df.set_index("FIPS")["NAME"].to_dict()
    url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
    geo_data = fetch_geojson(url)
    rows = []
    for feat in geo_data["features"]:
        fips = feat["id"]
        geom = shape(feat["geometry"])
        pop_val = county_pop_dict.get(fips, "No data")
        name_val = county_name_dict.get(fips, "No data")
        rows.append({
            "geometry": geom,
            "FIPS": fips,
            "NAME": name_val,
            "POP": pop_val
        })
    gdf = gpd.GeoDataFrame(rows, crs="EPSG:4326")
    gdf["geometry"] = gdf["geometry"].simplify(tolerance=0.01, preserve_topology=True)
    return gdf

In [8]:
state_abbrev_to_fips = {
    "AL": "01", "AK": "02", "AZ": "04", "AR": "05", "CA": "06", "CO": "08",
    "CT": "09", "DE": "10", "DC": "11", "FL": "12", "GA": "13", "HI": "15",
    "ID": "16", "IL": "17", "IN": "18", "IA": "19", "KS": "20", "KY": "21",
    "LA": "22", "ME": "23", "MD": "24", "MA": "25", "MI": "26", "MN": "27",
    "MS": "28", "MO": "29", "MT": "30", "NE": "31", "NV": "32", "NH": "33",
    "NJ": "34", "NM": "35", "NY": "36", "NC": "37", "ND": "38", "OH": "39",
    "OK": "40", "OR": "41", "PA": "42", "RI": "44", "SC": "45", "SD": "46",
    "TN": "47", "TX": "48", "UT": "49", "VT": "50", "VA": "51", "WA": "53",
    "WV": "54", "WI": "55", "WY": "56"
}

In [9]:
def load_and_merge_caps(_states_gdf):
    caps_df = pd.read_csv("caps_plans.csv")
    caps_df["State"] = caps_df["State"].str.strip().str.upper()
    caps_df["STATE_FIPS"] = caps_df["State"].map(state_abbrev_to_fips)
    caps_df["plan_info"] = caps_df.apply(
        lambda row: f"{row['City']}, {row['Year']}, {row['Plan Type']}", axis=1
    )
    grouped = caps_df.groupby("STATE_FIPS").agg(
        n_caps=("Plan Type", "count"),
        plan_list=("plan_info", lambda x: list(x))
    ).reset_index()
    merged = _states_gdf.merge(grouped, on="STATE_FIPS", how="left")
    merged["n_caps"] = merged["n_caps"].fillna(0).astype(int)
    merged["plan_list"] = merged["plan_list"].apply(lambda x: x if isinstance(x, list) else [])
    return merged

In [10]:
def load_and_merge_caps_county(_counties_gdf):
    caps_df = pd.read_csv("caps_plans.csv")
    mapping_df = pd.read_csv("city_county_mapping.csv")
    # Standardize text for matching
    caps_df["State"] = caps_df["State"].str.strip().str.upper()
    mapping_df["CountyKey"] = mapping_df["CountyName"].apply(
        lambda x: x.upper().split(',')[0].replace(" COUNTY", "").strip()
    )
    merged_caps = pd.merge(
        caps_df, mapping_df, 
        left_on=["City", "State"], 
        right_on=["CityName", "StateName"], 
        how="left"
    )
    merged_caps["plan_info"] = merged_caps.apply(
        lambda row: f"{row['City']}, {row['Year']}, {row['Plan Type']}", axis=1
    )
    merged_caps["CountyKey"] = merged_caps["CountyName"].apply(
        lambda x: x.upper().split(',')[0].replace(" COUNTY", "").strip() if pd.notnull(x) else None
    )
    grouped = merged_caps.groupby(["CountyKey", "StateName"]).agg(
        n_caps=("Plan Type", "count"),
        plan_list=("plan_info", lambda x: list(x))
    ).reset_index()
    fips_to_abbrev = {v: k for k, v in state_abbrev_to_fips.items()}
    _counties_gdf["STATE"] = _counties_gdf["FIPS"].str[:2].map(fips_to_abbrev)
    _counties_gdf["CountyKey"] = _counties_gdf["NAME"].apply(
        lambda x: x.upper().split(',')[0].replace(" COUNTY", "").strip()
    )
    merged_counties = _counties_gdf.merge(
        grouped, 
        left_on=["CountyKey", "STATE"], 
        right_on=["CountyKey", "StateName"], 
        how="left"
    )
    merged_counties["n_caps"] = merged_counties["n_caps"].fillna(0).astype(int)
    merged_counties["plan_list"] = merged_counties["plan_list"].apply(lambda x: x if isinstance(x, list) else [])
    return merged_counties

In [11]:
def load_city_mapping():
    """Load the city mapping CSV for marker locations."""
    df = pd.read_csv("city_county_mapping.csv")
    df["CityName"] = df["CityName"].str.strip().str.upper()
    df["StateName"] = df["StateName"].str.strip().str.upper()
    df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
    df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")
    return df

In [12]:
def load_city_plans():
    """Load and group caps_plans data by city and state."""
    df = pd.read_csv("caps_plans.csv")
    df["City"] = df["City"].str.strip().str.upper()
    df["State"] = df["State"].str.strip().str.upper()
    df["plan_info"] = df.apply(lambda row: f"{row['Year']}, {row['Plan Type']}", axis=1)
    grouped = df.groupby(["City", "State"]).agg(plan_list=("plan_info", lambda x: list(x))).reset_index()
    return grouped

In [13]:
state_df = fetch_state_data()
county_df = fetch_county_data()

states_gdf = build_states_gdf(state_df, state_abbrev_to_fips)
counties_gdf = build_counties_gdf(county_df)

In [14]:
state_df.head()

,NAME,POP,state
0,Alabama,4903185,01
1,Alaska,731545,02
2,Arizona,7278717,04
3,Arkansas,3017804,05
4,California,39512223,06


In [15]:
county_df.head()

,NAME,POP,state,county,countyName,stateName,FIPS
0,"Fayette County, Illinois",21336,17,051,Fayette County,Illinois,17051
1,"Logan County, Illinois",28618,17,107,Logan County,Illinois,17107
2,"Massac County, Illinois",13772,17,127,Massac County,Illinois,17127
3,"Saline County, Illinois",23491,17,165,Saline County,Illinois,17165
4,"Cass County, Illinois",12147,17,017,Cass County,Illinois,17017


In [16]:
states_gdf.head()

,geometry,STATE_FIPS,NAME,POP
0,"POLYGON ((-87.3593 35.00118, -85.60668 34.9847...",01,Alabama,4903185
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",02,Alaska,731545
2,"POLYGON ((-109.0425 37.00026, -109.04798 31.33...",04,Arizona,7278717
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",05,Arkansas,3017804
4,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",06,California,39512223


In [17]:
counties_gdf.head()

,geometry,FIPS,NAME,POP
0,"POLYGON ((-86.49677 32.34444, -86.7179 32.4028...",01001,"Autauga County, Alabama",55869
1,"POLYGON ((-86.5778 33.76532, -86.75914 33.8406...",01009,"Blount County, Alabama",57826
2,"POLYGON ((-85.18413 32.87052, -85.12342 32.772...",01017,"Chambers County, Alabama",33254
3,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",01021,"Chilton County, Alabama",44428
4,"POLYGON ((-88.13999 34.5817, -88.09789 34.8922...",01033,"Colbert County, Alabama",55241


In [18]:
states_gdf_caps =load_and_merge_caps(states_gdf)
states_gdf_caps.head()

,geometry,STATE_FIPS,NAME,POP,n_caps,plan_list
0,"POLYGON ((-87.3593 35.00118, -85.60668 34.9847...",01,Alabama,4903185,1,"[Birmingham, 2013, Green Plan]"
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",02,Alaska,731545,1,"[Anchorage, 2019, Mitigation Primary CAP]"
2,"POLYGON ((-109.0425 37.00026, -109.04798 31.33...",04,Arizona,7278717,2,"[Glendale, 2011, Green Plan, Phoenix, 2021, Mi..."
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",05,Arkansas,3017804,0,[]
4,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",06,California,39512223,7,"[Oakland, 2020, Mitigation Primary CAP, Los An..."


In [19]:
counties_gdf_caps = load_and_merge_caps_county(counties_gdf)
counties_gdf_caps.head()

,geometry,FIPS,NAME,POP,STATE,CountyKey,StateName,n_caps,plan_list
0,"POLYGON ((-86.49677 32.34444, -86.7179 32.4028...",01001,"Autauga County, Alabama",55869,AL,AUTAUGA,NaN,0,[]
1,"POLYGON ((-86.5778 33.76532, -86.75914 33.8406...",01009,"Blount County, Alabama",57826,AL,BLOUNT,NaN,0,[]
2,"POLYGON ((-85.18413 32.87052, -85.12342 32.772...",01017,"Chambers County, Alabama",33254,AL,CHAMBERS,NaN,0,[]
3,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",01021,"Chilton County, Alabama",44428,AL,CHILTON,NaN,0,[]
4,"POLYGON ((-88.13999 34.5817, -88.09789 34.8922...",01033,"Colbert County, Alabama",55241,AL,COLBERT,NaN,0,[]


In [21]:
city_mapping_df = load_city_mapping()
city_plans_df = load_city_plans()

In [22]:
city_mapping_df.head()

,CityName,StateName,CountyName,Latitude,Longitude
0,ALAMEDA,CA,"Alameda County, California",37.7652,-122.2416
1,ALBUQUERQUE,NM,"Bernalillo County, New Mexico",35.0844,-106.6504
2,ALEXANDRIA,VA,"Alexandria City, Virginia",38.8048,-77.0469
3,ANAHEIM,CA,"Orange County, California",33.8366,-117.9143
4,ANCHORAGE,AK,"Anchorage Municipality, Alaska",61.2181,-149.9003


In [23]:
city_plans_df.head()

,City,State,plan_list
0,ALAMEDA,CA,"[2019, Equal Adaptation-Mitigation Plan]"
1,ALBUQUERQUE,NM,"[2021, Green Plan]"
2,ALEXANDRIA,VA,"[2019, Mitigation Only CAP]"
3,ANAHEIM,CA,"[2020, Mitigation Plan]"
4,ANCHORAGE,AK,"[2019, Mitigation Primary CAP]"


In [24]:
nri_df = pd.read_excel("data/NRI Future Risk Index.xlsx")

nri_df.head()


,OBJECTID,NRI_ID,STATE,STATEABBRV,COUNTY,STCOFIPS,CFLD_EALT,CFLD_EALR,CFLD_RISKV,CFLD_RISKR,...,WFIR_LATE_LOWER_HM,WFIR_LATE_LOWER_PRISKS,WFIR_LATE_LOWER_PRISKR,WFIR_LATE_LOWER_PALR,WFIR_LATE_LOWER_DELTAR,WFIR_LATE_HIGHER_HM,WFIR_LATE_HIGHER_PRISKS,WFIR_LATE_HIGHER_PRISKR,WFIR_LATE_HIGHER_PALR,WFIR_LATE_HIGHER_DELTAR
0,1,C01001,Alabama,AL,Autauga,1001,NaN,Not Applicable,NaN,Not Applicable,...,1.619215,27.90,Very Low,Very Low (<85.9K),Relatively Moderate (1.57x - 1.82x),2.396241,26.25,Very Low,Very Low (<85.9K),Very High (>2.11x)
1,2,C01003,Alabama,AL,Baldwin,1003,222929.4057,Relatively Low (55K - 932K),196783.4067,Relatively Low,...,1.433374,92.78,Relatively Moderate,Relatively Moderate (842K - 5.88M),Relatively Low (1.38x - 1.57x),1.862422,92.78,Relatively Moderate,Relatively Moderate (842K - 5.88M),Relatively High (1.82x - 2.11x)
2,3,C01005,Alabama,AL,Barbour,1005,NaN,Not Applicable,NaN,Not Applicable,...,1.461678,22.88,Very Low,Very Low (<85.9K),Relatively Low (1.38x - 1.57x),2.196657,21.92,Very Low,Very Low (<85.9K),Very High (>2.11x)
3,4,C01007,Alabama,AL,Bibb,1007,NaN,Not Applicable,NaN,Not Applicable,...,1.631516,13.78,Very Low,Very Low (<85.9K),Relatively Moderate (1.57x - 1.82x),2.525198,13.52,Very Low,Very Low (<85.9K),Very High (>2.11x)
4,5,C01009,Alabama,AL,Blount,1009,NaN,Not Applicable,NaN,Not Applicable,...,1.573867,17.34,Very Low,Very Low (<85.9K),Relatively Moderate (1.57x - 1.82x),2.536543,17.63,Very Low,Very Low (<85.9K),Very High (>2.11x)


In [26]:
nri_df = nri_df[["STATEABBRV", "STATE", "COUNTY", "STCOFIPS", 
                     "CFLD_MID_HIGHER_PRISKS", 
                     "CFLD_LATE_HIGHER_PRISKS",
                     "CFLD_MID_HIGHER_HM",
                     "CFLD_LATE_HIGHER_HM",
                     "WFIR_MID_HIGHER_PRISKS",
                     "WFIR_LATE_HIGHER_PRISKS",
                     "WFIR_MID_HIGHER_HM",
                     "WFIR_LATE_HIGHER_HM",
                     "DRGT_MID_HIGHER_PRISKS",
                     "DRGT_LATE_HIGHER_PRISKS",
                     "DRGT_MID_HIGHER_HM",
                     "DRGT_LATE_HIGHER_HM",
                     "HRCN_MID_HIGHER_PRISKS",
                     "HRCN_LATE_HIGHER_PRISKS",
                     "HRCN_MID_HIGHER_HM",
                     "HRCN_LATE_HIGHER_HM"]]

nri_df.head()

,STATEABBRV,STATE,COUNTY,STCOFIPS,CFLD_MID_HIGHER_PRISKS,CFLD_LATE_HIGHER_PRISKS,CFLD_MID_HIGHER_HM,CFLD_LATE_HIGHER_HM,WFIR_MID_HIGHER_PRISKS,WFIR_LATE_HIGHER_PRISKS,WFIR_MID_HIGHER_HM,WFIR_LATE_HIGHER_HM,DRGT_MID_HIGHER_PRISKS,DRGT_LATE_HIGHER_PRISKS,DRGT_MID_HIGHER_HM,DRGT_LATE_HIGHER_HM,HRCN_MID_HIGHER_PRISKS,HRCN_LATE_HIGHER_PRISKS,HRCN_MID_HIGHER_HM,HRCN_LATE_HIGHER_HM
0,AL,Alabama,Autauga,1001,NaN,NaN,NaN,NaN,26.54,26.25,1.459433,2.396241,55.36,51.61,3.030829,5.824792,68.15,69.13,2.074969,2.443176
1,AL,Alabama,Baldwin,1003,57.14,52.11,8.71153,11.657615,92.55,92.78,1.330216,1.862422,86.76,82.63,2.042401,4.228729,98.88,98.88,2.051148,2.361065
2,AL,Alabama,Barbour,1005,NaN,NaN,NaN,NaN,22.88,21.92,1.411380,2.196657,78.46,79.22,1.594787,5.012215,77.25,77.57,2.039212,2.260921
3,AL,Alabama,Bibb,1007,NaN,NaN,NaN,NaN,13.08,13.52,1.503687,2.525198,35.28,31.08,2.494643,4.429406,51.86,52.00,2.076363,2.478656
4,AL,Alabama,Blount,1009,NaN,NaN,NaN,NaN,17.56,17.63,1.547350,2.536543,53.13,50.33,2.172142,4.410612,54.69,55.18,2.056977,2.413120


In [27]:
grouped_states = nri_df.groupby("STATE").agg(
        CFLD_MID_HIGHER_PRISKS=("CFLD_MID_HIGHER_PRISKS", "mean"),
        CFLD_LATE_HIGHER_PRISKS=("CFLD_LATE_HIGHER_PRISKS", "mean"),
        CFLD_MID_HIGHER_HM=("CFLD_MID_HIGHER_HM", "mean"),
        CFLD_LATE_HIGHER_HM=("CFLD_LATE_HIGHER_HM", "mean"),
        WFIR_MID_HIGHER_PRISKS=("WFIR_MID_HIGHER_PRISKS", "mean"),
        WFIR_LATE_HIGHER_PRISKS=("WFIR_LATE_HIGHER_PRISKS", "mean"),
        WFIR_MID_HIGHER_HM=("WFIR_MID_HIGHER_HM", "mean"),
        WFIR_LATE_HIGHER_HM=("WFIR_LATE_HIGHER_HM", "mean"),
        DRGT_MID_HIGHER_PRISKS=("DRGT_MID_HIGHER_PRISKS", "mean"),
        DRGT_LATE_HIGHER_PRISKS=("DRGT_LATE_HIGHER_PRISKS", "mean"),
        DRGT_MID_HIGHER_HM=("DRGT_MID_HIGHER_HM", "mean"),
        DRGT_LATE_HIGHER_HM=("DRGT_LATE_HIGHER_HM", "mean"),
        HRCN_MID_HIGHER_PRISKS=("HRCN_MID_HIGHER_PRISKS", "mean"),
        HRCN_LATE_HIGHER_PRISKS=("HRCN_LATE_HIGHER_PRISKS", "mean"),
        HRCN_MID_HIGHER_HM=("HRCN_MID_HIGHER_HM", "mean"),
        HRCN_LATE_HIGHER_HM=("HRCN_LATE_HIGHER_HM", "mean"),
    )

grouped_states.head()

,CFLD_MID_HIGHER_PRISKS,CFLD_LATE_HIGHER_PRISKS,CFLD_MID_HIGHER_HM,CFLD_LATE_HIGHER_HM,WFIR_MID_HIGHER_PRISKS,WFIR_LATE_HIGHER_PRISKS,WFIR_MID_HIGHER_HM,WFIR_LATE_HIGHER_HM,DRGT_MID_HIGHER_PRISKS,DRGT_LATE_HIGHER_PRISKS,DRGT_MID_HIGHER_HM,DRGT_LATE_HIGHER_HM,HRCN_MID_HIGHER_PRISKS,HRCN_LATE_HIGHER_PRISKS,HRCN_MID_HIGHER_HM,HRCN_LATE_HIGHER_HM
STATE,,,,,,,,,,,,,,,,
Alabama,32.796000,31.668000,2.554327,4.684517,29.166104,28.964627,1.480740,2.385828,55.304030,52.645224,2.115369,4.607729,70.734478,71.190149,2.052843,2.364846
Alaska,16.510577,16.115962,0.000000,0.000000,36.186888,35.933948,1.304417,1.986042,7.650100,7.647767,1.301138,1.539081,NaN,NaN,0.000000,0.000000
American Samoa,NaN,NaN,14.870773,370.744499,NaN,NaN,1.131023,1.395699,NaN,NaN,2.422716,5.524657,NaN,NaN,0.000000,0.000000
Arizona,NaN,NaN,NaN,NaN,89.348667,88.086000,1.168167,1.339359,83.332000,80.008667,3.894203,6.664982,17.741000,16.910091,0.000000,0.000000
Arkansas,NaN,NaN,NaN,NaN,41.452787,41.118347,1.605701,2.478493,53.402067,52.375267,2.070068,5.258882,41.906667,44.248133,1.762344,3.031242


In [30]:
def merge_nri_data(states_gdf_caps, counties_gdf_caps):
    nri_df = pd.read_excel("data/NRI Future Risk Index.xlsx")
    nri_df = nri_df[["STATEABBRV", "STATE", "COUNTY", "STCOFIPS", 
                        "CFLD_MID_HIGHER_PRISKS", 
                        "CFLD_LATE_HIGHER_PRISKS",
                        "CFLD_MID_HIGHER_HM",
                        "CFLD_LATE_HIGHER_HM",
                        "WFIR_MID_HIGHER_PRISKS",
                        "WFIR_LATE_HIGHER_PRISKS",
                        "WFIR_MID_HIGHER_HM",
                        "WFIR_LATE_HIGHER_HM",
                        "DRGT_MID_HIGHER_PRISKS",
                        "DRGT_LATE_HIGHER_PRISKS",
                        "DRGT_MID_HIGHER_HM",
                        "DRGT_LATE_HIGHER_HM",
                        "HRCN_MID_HIGHER_PRISKS",
                        "HRCN_LATE_HIGHER_PRISKS",
                        "HRCN_MID_HIGHER_HM",
                        "HRCN_LATE_HIGHER_HM"]]

    
    grouped_states = nri_df.groupby("STATE").agg(
        CFLD_MID_HIGHER_PRISKS=("CFLD_MID_HIGHER_PRISKS", "mean"),
        CFLD_LATE_HIGHER_PRISKS=("CFLD_LATE_HIGHER_PRISKS", "mean"),
        CFLD_MID_HIGHER_HM=("CFLD_MID_HIGHER_HM", "mean"),
        CFLD_LATE_HIGHER_HM=("CFLD_LATE_HIGHER_HM", "mean"),
        WFIR_MID_HIGHER_PRISKS=("WFIR_MID_HIGHER_PRISKS", "mean"),
        WFIR_LATE_HIGHER_PRISKS=("WFIR_LATE_HIGHER_PRISKS", "mean"),
        WFIR_MID_HIGHER_HM=("WFIR_MID_HIGHER_HM", "mean"),
        WFIR_LATE_HIGHER_HM=("WFIR_LATE_HIGHER_HM", "mean"),
        DRGT_MID_HIGHER_PRISKS=("DRGT_MID_HIGHER_PRISKS", "mean"),
        DRGT_LATE_HIGHER_PRISKS=("DRGT_LATE_HIGHER_PRISKS", "mean"),
        DRGT_MID_HIGHER_HM=("DRGT_MID_HIGHER_HM", "mean"),
        DRGT_LATE_HIGHER_HM=("DRGT_LATE_HIGHER_HM", "mean"),
        HRCN_MID_HIGHER_PRISKS=("HRCN_MID_HIGHER_PRISKS", "mean"),
        HRCN_LATE_HIGHER_PRISKS=("HRCN_LATE_HIGHER_PRISKS", "mean"),
        HRCN_MID_HIGHER_HM=("HRCN_MID_HIGHER_HM", "mean"),
        HRCN_LATE_HIGHER_HM=("HRCN_LATE_HIGHER_HM", "mean"),
    )
    counties_gdf_caps['FIPS'] = pd.to_numeric(counties_gdf_caps['FIPS'], errors='coerce').fillna(0).astype(int)
    nri_df['STCOFIPS'] = pd.to_numeric(nri_df['STCOFIPS'], errors='coerce').fillna(0).astype(int)

    merged_states_gdf = states_gdf_caps.merge(grouped_states, left_on="NAME", right_on="STATE", how="left")
    merged_counties_gdf = counties_gdf_caps.merge(nri_df, left_on="FIPS", right_on="STCOFIPS", how="left")
    return merged_states_gdf, merged_counties_gdf

merged_states_gdf, merged_counties_gdf = merge_nri_data(states_gdf_caps, counties_gdf_caps)
merged_states_gdf.head()

,geometry,STATE_FIPS,NAME,POP,n_caps,plan_list,CFLD_MID_HIGHER_PRISKS,CFLD_LATE_HIGHER_PRISKS,CFLD_MID_HIGHER_HM,CFLD_LATE_HIGHER_HM,...,WFIR_MID_HIGHER_HM,WFIR_LATE_HIGHER_HM,DRGT_MID_HIGHER_PRISKS,DRGT_LATE_HIGHER_PRISKS,DRGT_MID_HIGHER_HM,DRGT_LATE_HIGHER_HM,HRCN_MID_HIGHER_PRISKS,HRCN_LATE_HIGHER_PRISKS,HRCN_MID_HIGHER_HM,HRCN_LATE_HIGHER_HM
0,"POLYGON ((-87.3593 35.00118, -85.60668 34.9847...",01,Alabama,4903185,1,"[Birmingham, 2013, Green Plan]",32.796000,31.668000,2.554327,4.684517,...,1.480740,2.385828,55.304030,52.645224,2.115369,4.607729,70.734478,71.190149,2.052843,2.364846
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",02,Alaska,731545,1,"[Anchorage, 2019, Mitigation Primary CAP]",16.510577,16.115962,0.000000,0.000000,...,1.304417,1.986042,7.650100,7.647767,1.301138,1.539081,NaN,NaN,0.000000,0.000000
2,"POLYGON ((-109.0425 37.00026, -109.04798 31.33...",04,Arizona,7278717,2,"[Glendale, 2011, Green Plan, Phoenix, 2021, Mi...",NaN,NaN,NaN,NaN,...,1.168167,1.339359,83.332000,80.008667,3.894203,6.664982,17.741000,16.910091,0.000000,0.000000
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",05,Arkansas,3017804,0,[],NaN,NaN,NaN,NaN,...,1.605701,2.478493,53.402067,52.375267,2.070068,5.258882,41.906667,44.248133,1.762344,3.031242
4,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",06,California,39512223,7,"[Oakland, 2020, Mitigation Primary CAP, Los An...",64.382143,67.408810,28.711658,128.632815,...,1.135821,1.233403,89.078983,87.950707,2.971937,6.388950,19.233333,18.393333,0.000000,0.000000


In [31]:
merged_counties_gdf.head()

,geometry,FIPS,NAME,POP,STATE_x,CountyKey,StateName,n_caps,plan_list,STATEABBRV,...,WFIR_MID_HIGHER_HM,WFIR_LATE_HIGHER_HM,DRGT_MID_HIGHER_PRISKS,DRGT_LATE_HIGHER_PRISKS,DRGT_MID_HIGHER_HM,DRGT_LATE_HIGHER_HM,HRCN_MID_HIGHER_PRISKS,HRCN_LATE_HIGHER_PRISKS,HRCN_MID_HIGHER_HM,HRCN_LATE_HIGHER_HM
0,"POLYGON ((-86.49677 32.34444, -86.7179 32.4028...",1001,"Autauga County, Alabama",55869,AL,AUTAUGA,NaN,0,[],AL,...,1.459433,2.396241,55.36,51.61,3.030829,5.824792,68.15,69.13,2.074969,2.443176
1,"POLYGON ((-86.5778 33.76532, -86.75914 33.8406...",1009,"Blount County, Alabama",57826,AL,BLOUNT,NaN,0,[],AL,...,1.547350,2.536543,53.13,50.33,2.172142,4.410612,54.69,55.18,2.056977,2.413120
2,"POLYGON ((-85.18413 32.87052, -85.12342 32.772...",1017,"Chambers County, Alabama",33254,AL,CHAMBERS,NaN,0,[],AL,...,1.457237,2.394632,35.48,35.48,1.537855,4.445414,70.08,70.30,2.032321,2.255343
3,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",1021,"Chilton County, Alabama",44428,AL,CHILTON,NaN,0,[],AL,...,1.488076,2.470998,47.98,43.56,2.714399,4.961928,62.31,62.76,2.078682,2.500578
4,"POLYGON ((-88.13999 34.5817, -88.09789 34.8922...",1033,"Colbert County, Alabama",55241,AL,COLBERT,NaN,0,[],AL,...,1.620790,2.728143,37.99,37.29,2.080031,5.361639,55.72,56.44,2.060532,2.455989
